# 서울 북촌 지역 유동인구 지도 시각화

데이터 출처 : [서울 열린데이터 광장](http://data.seoul.go.kr/dataList/OA-15766/S/1/datasetView.do;jsessionid=5A575C621AF40C62FA5BC7E870265E6E.new_portal-svr-21)

### 1. 데이터 전처리

In [1]:
import pandas as pd
import numpy as np

df=pd.read_csv('서울시 북촌 CCTV 유동인구 일간_주간_월간 수집 정보.csv')
df.head()

,기준날짜,카메라코드,카메라명칭,설명,계수시작시간,계수끝시간,진입계수,진출계수,주간 진입계수,주간 진출계수,월간 진입계수,월간 진출계수,데이터입력시간
0,20190929,2,계동길 69,계동교회 앞,40:37.4,50:37.4,13566,14464,NaN,NaN,NaN,NaN,50:37.4
1,20190929,4,북촌로5가길 38,삼청파출소 사잇길,40:37.4,50:37.4,6520,7662,NaN,NaN,NaN,NaN,50:37.4
2,20190929,1,율곡로3길 50,덕성여고 앞,40:37.4,50:37.4,11257,13187,NaN,NaN,NaN,NaN,50:37.4
3,20190929,3,북촌로11길 1,돈미약국 앞,40:37.4,50:37.4,8761,7776,NaN,NaN,NaN,NaN,50:37.4
4,20190928,1,율곡로3길 50,덕성여고 앞,40:37.8,50:37.9,11710,13630,NaN,NaN,NaN,NaN,50:37.9


In [2]:
df.sort_values(by='기준날짜',inplace=True)
df = df.reset_index(drop=True)

In [3]:
new = df[['기준날짜','카메라명칭']].copy()
new['계수 평균'] = (df['진입계수'] + df['진출계수'])/2
new.head()

,기준날짜,카메라명칭,계수 평균
0,20190928,율곡로3길 50,12670.0
1,20190928,북촌로11길 1,9436.0
2,20190928,북촌로5가길 38,8253.0
3,20190928,계동길 69,17308.0
4,20190929,계동길 69,14015.0


In [4]:
location = pd.DataFrame(new['카메라명칭'].unique(), columns=['카메라명칭'])
location['위도'] = pd.Series([37.57777, 37.581425, 37.581286, 37.580543])
location['경도'] = pd.Series([126.982634, 126.984794, 126.981567, 126.986594])
location

,카메라명칭,위도,경도
0,율곡로3길 50,37.577770,126.982634
1,북촌로11길 1,37.581425,126.984794
2,북촌로5가길 38,37.581286,126.981567
3,계동길 69,37.580543,126.986594


In [5]:
new = pd.merge(new, location, on='카메라명칭')
new = new.sort_values(by='기준날짜')
new.head()

,기준날짜,카메라명칭,계수 평균,위도,경도
0,20190928,율곡로3길 50,12670.0,37.577770,126.982634
87,20190928,계동길 69,17308.0,37.580543,126.986594
58,20190928,북촌로5가길 38,8253.0,37.581286,126.981567
29,20190928,북촌로11길 1,9436.0,37.581425,126.984794
88,20190929,계동길 69,14015.0,37.580543,126.986594


In [15]:
new = new.set_index('기준날짜')
new.to_csv('북촌 유동인구.csv', encoding='EUC-KR')

----------------------------------------------

### 2. 시각화

In [16]:
value = new['계수 평균'].values
minimum = min(value)
maximum = max(value)
new['계수 평균'] = new['계수 평균'].transform(func = lambda x : 300*(x - minimum)/(maximum-minimum))
new['계수 평균'] = np.ceil(new['계수 평균']).astype(int)
new.head()

,카메라명칭,계수 평균,위도,경도
기준날짜,,,,
20190928,율곡로3길 50,134,37.577770,126.982634
20190928,계동길 69,192,37.580543,126.986594
20190928,북촌로5가길 38,80,37.581286,126.981567
20190928,북촌로11길 1,94,37.581425,126.984794
20190929,계동길 69,151,37.580543,126.986594


In [17]:
date = list(new.index.unique())

In [19]:
matrix = []

for j in date:
    a = []
    temp = new[new.index==j]
    
    for lat, lon, value in zip(temp['위도'], temp['경도'], temp['계수 평균']):
        [a.append([lat+np.random.normal(0,0.0003), lon+np.random.normal(0,0.0003)]) for i in range(value)]
    
    matrix.append(a)

In [20]:
def change_date(x):
    x = str(x)
    return x[:4] + '-' + x[4:6] + '-' + x[6:]

time_ind = list(map(lambda i: change_date(i), date))
print(time_ind)

['2019-09-28', '2019-09-29', '2019-09-30', '2019-10-01', '2019-10-02', '2019-10-03', '2019-10-04', '2019-10-05', '2019-10-06', '2019-10-07', '2019-10-08', '2019-10-09', '2019-10-10', '2019-10-11', '2019-10-12', '2019-10-13', '2019-10-14', '2019-10-15', '2019-10-16', '2019-10-17', '2019-10-18', '2019-10-19', '2019-10-20', '2019-10-21', '2019-10-22', '2019-10-23', '2019-10-24', '2019-10-25', '2019-10-26']


In [21]:
import folium
import folium.plugins as plugins

m = folium.Map([37.579362, 126.984746], zoom_start=16)
plugins.HeatMapWithTime(matrix, index=time_ind).add_to(m)
m